### **Modeling Development and Tracking Experiments with mlflow**

In [1]:
# Import Libraries

import pandas as pd
import numpy as np

import math
from scipy.stats import randint, uniform
from datetime import datetime

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from xgboost import XGBRegressor
import xgboost as xgb
from sklearn.ensemble import RandomForestRegressor,GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn import metrics

import seaborn as sns

import pickle
from pickle import dump
import os

import warnings
warnings.filterwarnings('ignore')

In [2]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

In [3]:
import mlflow
import mlflow.sklearn
import mlflow.pyfunc

In [4]:
mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("Laptop Prices Prediction")

<Experiment: artifact_location='file:///c:/Users/ADMIN/Desktop/Predict-Laptop-Prices-Streamlit/notebooks/mlruns/2', creation_time=1684007265560, experiment_id='2', last_update_time=1684007265560, lifecycle_stage='active', name='Laptop Prices Prediction', tags={}>

In [5]:
# Load data

df = pd.read_csv('..\data\cleaned_data2.csv')
df.head()

,MRP,RAM_Size,RAM_Type,Display,Processor,Storage,OS,Brand
0,36990,8,DDR4,14.0,Intel Core i3,256 GB SSD,Windows,Lenovo
1,39990,8,DDR4,15.6,Intel Core i3,512 GB SSD,Windows,Lenovo
2,32990,8,DDR4,15.6,Intel Core i3,512 GB SSD,Windows,ASUS
3,49990,8,DDR4,15.6,AMD Ryzen 5 Hexa Core,512 GB SSD,Windows,HP
4,49990,8,DDR4,15.6,Intel Core i5,512 GB SSD,Windows,ASUS


In [6]:
df.shape

(423, 8)

In [7]:
# Generate dictionaries of the categorical columns

def generate_dicts(df):
    categorical_cols = df.select_dtypes(include=['object']).columns
    dicts_cols = {}
    for col in categorical_cols:
        unique_values = df[col].unique()
        dict_col = {}
        for i, value in enumerate(sorted(unique_values)):
            dict_col[value] = i
        dicts_cols[col] = dict_col
    return dicts_cols

dicts_cols = generate_dicts(df)
print(dicts_cols)

# Replace the string values with the corresponding integer values

df = df.replace(dicts_cols)

{'RAM_Type': {'DDR4': 0, 'DDR5': 1, 'LPDDR3': 2, 'LPDDR4': 3, 'LPDDR4X': 4, 'LPDDR5': 5, 'Unified Memory': 6}, 'Processor': {'AMD Athlon Dual Core': 0, 'AMD Ryzen 3': 1, 'AMD Ryzen 3 Dual Core': 2, 'AMD Ryzen 3 Hexa Core': 3, 'AMD Ryzen 3 Quad Core': 4, 'AMD Ryzen 5': 5, 'AMD Ryzen 5 Dual Core': 6, 'AMD Ryzen 5 Hexa Core': 7, 'AMD Ryzen 5 Quad Core': 8, 'AMD Ryzen 7 Octa Core': 9, 'AMD Ryzen 7 Quad Core': 10, 'AMD Ryzen 9 Octa Core': 11, 'Intel Celeron Dual Core': 12, 'Intel Celeron Quad Core': 13, 'Intel Core i3': 14, 'Intel Core i5': 15, 'Intel Core i7': 16, 'Intel Core i9': 17, 'Intel Evo Core i5': 18, 'Intel Pentium Quad Core': 19, 'Intel Pentium Silver': 20, 'M1': 21, 'M1 Max': 22, 'M1 Pro': 23, 'M2': 24, 'Qualcomm Snapdragon 7c Gen 2': 25}, 'Storage': {'1 TB HDD': 0, '1 TB HDD, 128 GB SSD': 1, '1 TB HDD, 256 GB SSD': 2, '1 TB HDD, 512 GB SSD': 3, '128 GB SSD': 4, '256 GB SSD': 5, '512 GB SSD': 6}, 'OS': {'Chrome': 0, 'DOS': 1, 'Mac OS': 2, 'Windows': 3}, 'Brand': {'ALIENWARE': 0,

In [8]:
# Separate the target variable from the features

X = df.drop(columns=['MRP'], axis=1)
y = np.log(df['MRP'])

In [9]:
# Split data into train and test 

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [10]:
X_train_scaled = X_train.copy()
X_test_scaled = X_test.copy()

# Rescale RAM Size and Display columns only

scaler = MinMaxScaler()

X_train_scaled[['Display', 'RAM_Size']] = scaler.fit_transform(X_train_scaled[['Display', 'RAM_Size']])
X_test_scaled[['Display', 'RAM_Size']] = scaler.transform(X_test_scaled[['Display', 'RAM_Size']])

# Create new dataframes with the scaled data and original column names

X_train = pd.DataFrame(X_train_scaled, columns=X_train.columns)
X_test = pd.DataFrame(X_test_scaled, columns=X_test.columns)

### **Experiment Tracking**

**XGBoost Regressor**

In [19]:
with mlflow.start_run():

    mlflow.set_tag("algorithm", "XGBoost")

    # log the data for each run using log_param, log_metric, log_model
    mlflow.log_param("data-path", "..\data\cleaned_data2.csv")

    lr = 0.2
    mlflow.log_param("lr", lr)

    xgbr = XGBRegressor(learning_rate = lr)
    xgbr.fit(X_train, y_train)

    y_pred = xgbr.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test, y_pred)
    mlflow.log_metric("mse", mse)
    mlflow.log_metric("rmse", rmse)
    mlflow.log_metric("r2", r2)

    mlflow.sklearn.log_model(xgbr, artifact_path="models")
    
    mlflow.end_run()

2023/05/14 21:22:03 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: C:\Users\ADMIN\AppData\Local\Temp\tmp6pe0fsc5\model\model.pkl, flavor: sklearn), fall back to return ['scikit-learn==1.2.2', 'cloudpickle==2.2.1']. Set logging level to DEBUG to see the full traceback.


**GradientBoosting Regressor**

In [12]:
with mlflow.start_run():

    mlflow.set_tag("algorithm", "Gradient Boost")

    # log the data for each run using log_param, log_metric, log_model
    mlflow.log_param("data-path", "..\data\cleaned_data2.csv")
    
    lr = 0.25
    mlflow.log_param("lr", lr)
    
    gbr = GradientBoostingRegressor(learning_rate = lr)
    gbr.fit(X_train, y_train)

    y_pred = gbr.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test, y_pred)
    mlflow.log_metric("mse", mse)
    mlflow.log_metric("rmse", rmse)
    mlflow.log_metric("r2", r2)

    mlflow.sklearn.log_model(gbr, artifact_path="models")
    
    mlflow.end_run()

**Random Forest Regressor**

In [13]:
with mlflow.start_run():

    mlflow.set_tag("algorithm", "Random Forest")

    # log the data for each run using log_param, log_metric, log_model
    mlflow.log_param("data-path", "..\data\cleaned_data2.csv")

    n_estimators = 300
    mlflow.log_param("n_estimators", n_estimators)

    rf = RandomForestRegressor(n_estimators)
    rf.fit(X_train, y_train)

    y_pred = rf.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test, y_pred)
    mlflow.log_metric("mse", mse)
    mlflow.log_metric("rmse", rmse)
    mlflow.log_metric("r2", r2)
    
    mlflow.sklearn.log_model(rf, artifact_path="models")
    mlflow.end_run()

### **Hyperparameter Tuning with hyperopt**

In [22]:
mlflow.xgboost.autolog()

train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_test, label=y_test)

In [17]:
def objective(params):
    with mlflow.start_run():
        mlflow.set_tag("algorithm", "XGBoost")
        mlflow.log_params(params)
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=1000,
            evals=[(valid, 'validation')],
            early_stopping_rounds=50
        )
        y_pred = booster.predict(valid)
        rmse = mean_squared_error(y_test, y_pred, squared=False)
        mlflow.log_metric("mse", mse)
        mlflow.log_metric("rmse", rmse)
        mlflow.log_metric("r2", r2)
       
    return {'loss': rmse, 'status': STATUS_OK}

In [18]:
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 200, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:linear',
    'seed': 42
}

best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=50,
    trials=Trials()
)

[20:57:52] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-07593ffd91cd9da33-1\xgboost\xgboost-ci-windows\src\objective\regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation-rmse:3.98522                           
[1]	validation-rmse:1.52650                           
[2]	validation-rmse:0.65058                           
[3]	validation-rmse:0.38839                           
[4]	validation-rmse:0.32286                           
[5]	validation-rmse:0.30532                           
[6]	validation-rmse:0.30553                           
[7]	validation-rmse:0.29847                           
[8]	validation-rmse:0.29557                           
[9]	validation-rmse:0.29450                           
[10]	validation-rmse:0.29508                          
[11]	validation-rmse:0.29421                          
[12]	validation-rmse:0.28897                          
[13]	validation-rmse:0.28778                          

#### **Train the model with the attained hyperparameters**

In [24]:
mlflow.xgboost.autolog()

with mlflow.start_run():

    train = xgb.DMatrix(X_train, label=y_train)
    valid = xgb.DMatrix(X_test, label=y_test)

    best_params = {
    'learning_rate' : 0.33371037252980806,
    'max_depth' : 45,
    'min_child_weight' : 4.331442982301666,
    'objective': 'reg:linear',
    'reg_alpha' : 0.10673584255582483,
    'reg_lambda' : 0.18159812575093015,
    'seed' : 42
    }

    mlflow.log_params(best_params)

    booster = xgb.train(
        params=best_params,
        dtrain=train,
        num_boost_round=1000,
        evals=[(valid, 'validation')],
        early_stopping_rounds=50
    )

    y_pred = booster.predict(valid)
    rmse = mean_squared_error(y_test, y_pred, squared=False)
    mlflow.log_metric("rmse", rmse)
    mlflow.log_metric("mse", mse)
    mlflow.log_metric("r2", r2)

    with open("models/preprocessor.b", "wb") as f_out:
        pickle.dump(dv, f_out)
    mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

    mlflow.xgboost.log_model(booster, artifact_path="models_mlflow")

[21:40:33] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-07593ffd91cd9da33-1\xgboost\xgboost-ci-windows\src\objective\regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation-rmse:7.11903
[1]	validation-rmse:4.77215
[2]	validation-rmse:3.21424
[3]	validation-rmse:2.17457
[4]	validation-rmse:1.47758
[5]	validation-rmse:1.02172
[6]	validation-rmse:0.71468
[7]	validation-rmse:0.52218
[8]	validation-rmse:0.39970
[9]	validation-rmse:0.32347
[10]	validation-rmse:0.27519
[11]	validation-rmse:0.24691
[12]	validation-rmse:0.23429
[13]	validation-rmse:0.22477
[14]	validation-rmse:0.21979
[15]	validation-rmse:0.21839
[16]	validation-rmse:0.21540
[17]	validation-rmse:0.21348
[18]	validation-rmse:0.21354
[19]	validation-rmse:0.21180
[20]	validation-rmse:0.21121
[21]	validation-rmse:0.21017
[22]	validation-rmse:0.20918
[23]	validation-rmse:0.20801
[24]	validation-rmse:0.20770
[25]	validation-rmse:0.20754
[26]	validation-rmse:0.20

2023/05/14 21:43:49 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: C:\Users\ADMIN\AppData\Local\Temp\tmpms26zxm7\model, flavor: xgboost), fall back to return ['xgboost==1.7.5']. Set logging level to DEBUG to see the full traceback.
